# Convert weather station CSV files to parquet

In [1]:
import pytz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_columns(file, nlines):
    with open(file) as f:
        lines = [f.readline() for _ in range(nlines)]
        texts = [[text.strip(' .\n') for text in line.split('\t')] for line in lines]
        columns = ['_'.join(x).lstrip('_') for x in zip(*texts)]
    return columns

In [3]:
# ** weather data ***

files = ['2006.txt','2007.txt','2008.txt','2009.txt','2010.txt',
         '2011.txt','2012.txt','2013.txt','2014.txt','2015.txt',
         '2016.txt','2017.txt','2018.txt','2019.txt','2020.txt',
         'WXdata2021.txt','WXdata2022.txt','WXdata2023.txt',
         'WXdata-3-28-24.txt']

In [4]:
srcdir = 'data/src/wx'
dstdir = 'data/dst'

In [5]:
datasets = []

for file in files:
    
    fn = f'{srcdir}/{file}'
    
    columns = get_columns(fn, 2)
    raw = pd.read_csv(fn, skiprows=2, header=0, names=columns, parse_dates=False, na_values=['#VALUE!','---','------'], sep='\t', low_memory=False)

    # Time zone is unknown so we have only naive datetime or assume UTC,
    # 2008.txt contains duplicate datetimes at 2008-03-10 and 2008-05-07
    raw['Time'] = raw['Time'] + 'm'
    raw['DateTimeLocal'] = pd.to_datetime(raw['Date'] + ' ' + raw['Time'], format='%m/%d/%y %I:%M %p', utc=False)
    raw['DateTimeLocal'] = raw['DateTimeLocal'].dt.tz_localize('EST')

    # will convert the local to UTC in the DateTime later
    raw['DateTime'] = raw['DateTimeLocal']
    raw.drop(['Date', 'Time'], axis=1, inplace=True)

    # check for duplicates in the individual files
    print('Loaded:', fn, 'Removing duplicate DateTimes:', raw['DateTimeLocal'].duplicated().any())

    # not actually sure which of the duplicates we want
    df = raw.loc[~raw['DateTimeLocal'].duplicated(), :]
    
    datasets.append(df)

Loaded: data/src/wx/2006.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2007.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2008.txt Removing duplicate DateTimes: True
Loaded: data/src/wx/2009.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2010.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2011.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2012.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2013.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2014.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2015.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2016.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2017.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2018.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2019.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/2020.txt Removing duplicate DateTimes: False
Loaded: data/src/wx/WXdata

In [6]:
# concatenate the dataframes
wx_df = pd.concat(datasets)
wx_df.shape

(566870, 36)

In [7]:
# any DateTime duplicates after concatenation?
wx_df['DateTime'].duplicated().any()

False

In [8]:
# if no duplicates go ahead and set the frequency
wx_df.set_index(wx_df['DateTime'], inplace=True)
wx_df = wx_df.asfreq(freq='15min')
# wx_df = wx_df.resample('15min').first()

# update the index to UTC
wx_df.index = wx_df.index.tz_convert(pytz.utc)

In [9]:
wx_df.head()

,Temp_Out,Hi_Temp,Low_Temp,Out_Hum,Dew_Pt,Wind_Speed,Wind_Dir,Wind_Run,Hi_Speed,Hi_Dir,...,In_Hum,In_Dew,In_Heat,ET,Wind_Samp,Wind_Tx,ISS_Recept,Arc_Int,DateTimeLocal,DateTime
DateTime,,,,,,,,,,,,,,,,,,,,,
2006-06-09 01:45:00+00:00,62.9,63.2,62.8,96.0,61.7,0.0,S,0.0,2.0,SSE,...,60.0,59.9,75.1,0.0,345.0,1.0,100.0,15.0,2006-06-08 20:45:00-05:00,2006-06-08 20:45:00-05:00
2006-06-09 02:00:00+00:00,62.7,62.8,62.7,97.0,61.8,0.0,S,0.0,3.0,S,...,65.0,60.2,73.0,0.0,326.0,1.0,95.3,15.0,2006-06-08 21:00:00-05:00,2006-06-08 21:00:00-05:00
2006-06-09 02:15:00+00:00,62.5,62.7,62.5,97.0,61.6,0.0,SSE,0.0,2.0,S,...,66.0,59.6,71.8,0.0,345.0,1.0,100.0,15.0,2006-06-08 21:15:00-05:00,2006-06-08 21:15:00-05:00
2006-06-09 02:30:00+00:00,62.5,62.5,62.5,98.0,61.9,0.0,NaN,0.0,0.0,NaN,...,69.0,60.4,71.4,0.0,337.0,1.0,98.5,15.0,2006-06-08 21:30:00-05:00,2006-06-08 21:30:00-05:00
2006-06-09 02:45:00+00:00,62.4,62.5,62.4,98.0,61.8,0.0,NaN,0.0,0.0,NaN,...,73.0,61.5,71.2,0.0,348.0,1.0,100.0,15.0,2006-06-08 21:45:00-05:00,2006-06-08 21:45:00-05:00


In [10]:
wx_df.index

DatetimeIndex(['2006-06-09 01:45:00+00:00', '2006-06-09 02:00:00+00:00',
               '2006-06-09 02:15:00+00:00', '2006-06-09 02:30:00+00:00',
               '2006-06-09 02:45:00+00:00', '2006-06-09 03:00:00+00:00',
               '2006-06-09 03:15:00+00:00', '2006-06-09 03:30:00+00:00',
               '2006-06-09 03:45:00+00:00', '2006-06-09 04:00:00+00:00',
               ...
               '2024-03-28 10:30:00+00:00', '2024-03-28 10:45:00+00:00',
               '2024-03-28 11:00:00+00:00', '2024-03-28 11:15:00+00:00',
               '2024-03-28 11:30:00+00:00', '2024-03-28 11:45:00+00:00',
               '2024-03-28 12:00:00+00:00', '2024-03-28 12:15:00+00:00',
               '2024-03-28 12:30:00+00:00', '2024-03-28 12:45:00+00:00'],
              dtype='datetime64[ns, UTC]', name='DateTime', length=624237, freq='15min')

In [11]:
# write to parquet file
wx_df.to_parquet(f'{dstdir}/wx_data.parquet', index=True)